In [8]:
import sys, os

from torchvision.models import vit_b_16
import torch.nn as nn
import torch

from surgeon_pytorch import Extract, get_nodes

from PIL import Image
import torchvision.transforms as transforms
import os

import numpy as np

from matplotlib import pyplot as plt
import cv2

from captum.attr import LayerGradCam, LayerAttribution



In [9]:
external_repo_path = os.path.abspath('external/transformer_explainability')
if external_repo_path not in sys.path:
    sys.path.append(external_repo_path)

In [5]:
model = vit_b_16()
model.heads.head = nn.Linear(768,2)
model.load_state_dict(torch.load("logs/dfr_model.pth", map_location=torch.device('cpu'), weights_only=False))

<All keys matched successfully>

In [11]:
from external.transformer_explainability.baselines.ViT.ViT_LRP import vit_base_patch16_224 as vit_LRP
from external.transformer_explainability.baselines.ViT.ViT_explanation_generator import LRP

model2 = vit_LRP(pretrained=True)



Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth" to C:\Users\elmop/.cache\torch\hub\checkpoints\jx_vit_base_p16_224-80ecf9dd.pth


In [14]:
attribution_generator = LRP(model)


In [22]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def tensorize(img_path):
    image = Image.open(img_path).convert('RGB')
    return preprocess(image).unsqueeze(0)


def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return cam

def generate_visualization(original_image, class_index=None):
    transformer_attribution = attribution_generator.generate_LRP(original_image.unsqueeze(0).cuda(), method="transformer_attribution", index=class_index).detach()
    transformer_attribution = transformer_attribution.reshape(1, 1, 14, 14)
    transformer_attribution = torch.nn.functional.interpolate(transformer_attribution, scale_factor=16, mode='bilinear')
    transformer_attribution = transformer_attribution.reshape(224, 224).data.cpu().numpy()
    transformer_attribution = (transformer_attribution - transformer_attribution.min()) / (transformer_attribution.max() - transformer_attribution.min())



    image_transformer_attribution = original_image.permute(1, 2, 0).data.cpu().numpy()
    image_transformer_attribution = (image_transformer_attribution - image_transformer_attribution.min()) / (image_transformer_attribution.max() - image_transformer_attribution.min())
    vis = show_cam_on_image(image_transformer_attribution, transformer_attribution)
    vis =  np.uint8(255 * vis)
    vis = cv2.cvtColor(np.array(vis), cv2.COLOR_RGB2BGR)
    return vis

In [19]:
img = tensorize('notebooks/data/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg')
img.shape

torch.Size([1, 3, 224, 224])

In [20]:
out = model(img)

In [23]:
generate_visualization(img,0)

AssertionError: Torch not compiled with CUDA enabled